In [1]:
import pandas as pd
from libraries.db import DatabaseProxy

In [2]:
df = pd.read_csv("input/qa-pairs.csv", encoding="latin-1", dtype="str")
df.head()

,question,answer,partners_personnel_selected
0,Checking the financial stability of our suppli...,Our fiscal year begins in January and ends in ...,0
1,How many candidates do you have in your database?,"As of today, 4/25/2023, we have the following:...",0
2,How do you select which candidates are placed ...,Our specialized role known as the client speci...,0
3,How often do you review and refresh your datab...,Our sourcing platform is updated throughout th...,0
4,What type of background and drug testing do yo...,AccuSource is our service provider for backgro...,0


In [3]:
import re


non_alphanumeric = set()
pattern = r"[^a-zA-Z0-9\s]"

# get all non-alphanumeric characters in the questions and answers
for index, row in df.iterrows():
    question = row["question"]
    answer = str(row["answer"])
    non_alphanumeric.update(set(re.findall(pattern, question)))
    non_alphanumeric.update(set(re.findall(pattern, answer)))

In [4]:
non_alphanumeric

{'"',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 ':',
 '>',
 '?',
 '¥',
 'É',
 'Ð',
 'Ñ',
 'Ò',
 'Ó',
 'Õ'}

In [5]:
mapping = {"¥": "-", "É": "...", "Ð": "—", "Ñ": "—", "Ò": '"', "Ó": '"', "Õ": "'"}


def replace_non_alphanumeric(text):
    text = str(text)
    for key, value in mapping.items():
        text = text.replace(key, value)
    return text


df["question"] = df["question"].apply(replace_non_alphanumeric)
df["answer"] = df["answer"].apply(replace_non_alphanumeric)

In [6]:
db = DatabaseProxy(reset=True)

In [7]:
from libraries.db.model import Answer, Question


for _, row in df.iterrows():
    answer = Answer(text=str(row["answer"]).strip())
    answer = db.insert_answer(answer)
    question = Question(text=str(row["question"]).strip(), answer_id=answer.id)
    db.insert_question(question)

## View answers

In [8]:
db.get_answers()[:5]

[Answer(text='Our fiscal year begins in January and ends in December this contains four 5-week periods and eight 4-week periods with weeks ending on Sunday.', created_datetime=datetime.datetime(2023, 11, 17, 4, 27, 12), owner_organization=None, modified_datetime=datetime.datetime(2023, 11, 17, 4, 27, 12), id=1),
 Answer(text='As of today, 4/25/2023, we have the following:\n2060 associates who have successfully completed assignments and are available for work\n3464 associate who have been onboarded within the last 30 days and are available for work\n4117 associates going through the application process\nIn total we have 57083 associates active and unassigned in our system.', created_datetime=datetime.datetime(2023, 11, 17, 4, 27, 12), owner_organization=None, modified_datetime=datetime.datetime(2023, 11, 17, 4, 27, 12), id=2),
 Answer(text='Our specialized role known as the client specialist is a dedicated resource to our customers. Their primary role is to intake orders evaluate priori

In [9]:
db.get_answer_by_id(152)

Answer(text='Ex 1: We perform extensive screening for candidates that best match the job descriptions and are comfortable with the expectations at each site. We take care of all of our associates and over-communicate to ensure successful placement.                                                                                                                                                           \nEx 2: Employee Recognition\nWe are firm believers in recognizing and rewarding good behavior and have met with much success using some of these employee appreciation efforts \n-\tEmployee of the Month — working together with supervisory staff in identifying deserving associates each month.\n-\tRaffle Employee Attendance Program.  Anyone with 100% attendance with 40 hours plus in a week, raffle items (gas cards, gift cards, CASH). \n-\tEmployee "Goal" Lunch or Raffle:  When an employee or "Team" achieves a set goal or exceeds expectations, we would sponsor a special lunch or do a raffle fo

In [10]:
len(db.get_questions())

168